<a href="https://colab.research.google.com/github/YianKim/2022_uncertainty_aware_semisupervise/blob/main/Keras_UPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import PIL
import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import LSTM
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import *
from keras.models import load_model

from tqdm import tqdm

# cifar10 dataset

In [34]:
cifar10 = keras.datasets.cifar10 
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images/255
test_images = test_images/255

In [35]:
def dummy_labels(labels):
  zero_labels = np.zeros([labels.shape[0], 10], np.int8)  
  for i in range(labels.shape[0]):
    zero_labels[i][labels[i]] = 1
  return(zero_labels)

In [36]:
train_labels = dummy_labels(train_labels)
test_labels = dummy_labels(test_labels)

In [48]:
# 1000 labeled, 49000 unlabeled
indx = random.sample(range(train_labels.shape[0]),train_labels.shape[0])

lbl_train_images = train_images[indx[:1000]]
ubl_train_images = train_images[indx[1000:]]

lbl_train_labels = train_labels[indx[:1000]]
ubl_train_labels = train_labels[indx[1000:]]

In [116]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

def create_cnn_13():
  conv1a = Conv2D(128, 3, padding = 'same', activation='relu')
  bn1a = BatchNormalization()
  conv1b = Conv2D(128, 3, padding = 'same', activation='relu')
  bn1b = BatchNormalization()
  conv1c = Conv2D(128, 3, padding = 'same', activation='relu')
  bn1c = BatchNormalization()
  pl1 = MaxPooling2D(2, 2, padding = 'same')
  MCdrop1 = PermaDropout(0.5)

  conv2a = Conv2D(256, 3, padding = 'same', activation='relu')
  bn2a = BatchNormalization()
  conv2b = Conv2D(256, 3, padding = 'same', activation='relu')
  bn2b = BatchNormalization()
  conv2c = Conv2D(256, 3, padding = 'same', activation='relu')
  bn2c = BatchNormalization()
  pl2 = MaxPooling2D(2, 2, padding = 'same')
  MCdrop2 = PermaDropout(0.5)

  conv3a = Conv2D(512, 3, padding = 'same', activation='relu')
  bn3a = BatchNormalization()
  conv3b = Conv2D(256, 3, padding = 'same', activation='relu')
  bn3b = BatchNormalization()
  conv3c = Conv2D(128, 3, padding = 'same', activation='relu')
  bn3c = BatchNormalization()
  pl3 = AveragePooling2D(6, 2, padding = 'same')

  fc = Dense(10)
  activ = keras.layers.LeakyReLU(0.1)

  model = Sequential([
                      keras.Input(shape=(32, 32, 3)), 
                      conv1a, bn1a, activ,
                      conv1b, bn1b, activ,
                      conv1c, bn1c, activ,
                      pl1, MCdrop1,

                      conv2a, bn2a, activ,
                      conv2b, bn2b, activ,
                      conv2c, bn2c, activ,
                      pl2, MCdrop2,

                      conv3a, bn3a, activ,
                      conv3b, bn3b, activ,
                      conv3c, bn3c, activ,
                      pl3, Flatten(),
                      
                      fc
                      ])
  
  return model

def compile_cnn_13(model):

  adam = keras.optimizers.Adam(0.001)

  model.compile(
    optimizer = 'adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
  )

  return model

def cnn_13():

  model = create_cnn_13()
  model = compile_cnn_13(model)

  return model

def fit_and_labeling_cnn_13(Epoch, Batch):
  
  lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
  early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, mode='auto')

  model.fit(
      x=lbl_train_images,
      y=lbl_train_labels,
      epochs=Epoch,
      verbose=1,
      validation_split=0.2,
      batch_size=Batch,
      callbacks=[lr_reducer, early_stopper]
  )

  pred = model.predict(test_images)
  acc = (np.argmax(pred,axis=1) == np.argmax(test_labels,axis=1))*1
  print("test set 성능 : " + str(sum(acc)/len(acc)))

  for predsamples in tqdm(range(20)):
    if predsamples == 0 :
      predictions = model.predict(ubl_train_images)
    else:
      pred = model.predict(ubl_train_images)
      predictions = np.stack((predictions, pred))

  return predictions

In [ ]:
model = cnn_13()
fit_and_labeling_cnn_13(1000, 64)

Epoch 1/1000
